In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
data_dir = Path(os.environ["INPUT_DATA_DIR"])
transfer_input_files = data_dir.rglob("transfers-*.json")
asid_lookup_file = data_dir / "asidLookup.csv"

In [3]:
asid_lookup = pd.read_csv(asid_lookup_file)

In [4]:
date_cols = ["dateRequested", "dateCompleted"]
transfers = pd.concat((
    pd.read_json(f, convert_dates=date_cols)
    for f in transfer_input_files
))

In [5]:
supplier_transfers = transfers\
    .merge(asid_lookup, left_on='requestingPracticeAsid',right_on='ASID',how='left')\
    .merge(asid_lookup, suffixes=('_requesting', '_sending'), left_on='sendingPracticeAsid',right_on='ASID',how='left')\

supplier_transfers["MName_sending"] = supplier_transfers["MName_sending"].fillna("Unknown")
supplier_transfers["MName_requesting"] = supplier_transfers["MName_requesting"].fillna("Unknown")

In [6]:
supplier_transfers["MName_sending"].value_counts(dropna=False)

EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)    875743
THE PHOENIX PARTNERSHIP                         203366
IN PRACTICE SYSTEMS LTD                          24760
Unknown                                           3813
MICROTEST LTD                                      995
Name: MName_sending, dtype: int64

In [7]:
monthy_transfer_breakdown_by_supplier_pathways = supplier_transfers[
    ["dateRequested", "MName_requesting", "MName_sending", "status"]
].pivot_table(
    index=[
        supplier_transfers["dateRequested"].dt.year,
        supplier_transfers["dateRequested"].dt.month,
        "MName_requesting",
        "MName_sending"
    ],
    columns=["status"],
    aggfunc=len,
    fill_value=0
)
monthy_transfer_breakdown_by_supplier_pathways

dateRequested  \
status                                                                                                                TransferStatus.FAILED   
dateRequested dateRequested MName_requesting                             MName_sending                                                        
2020          9             EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                  2053   
                                                                         IN PRACTICE SYSTEMS LTD                                         58   
                                                                         MICROTEST LTD                                                   48   
                                                                         THE PHOENIX PARTNERSHIP                                       4207   
                                                                         Unknown                                                        104   
...                                                                                                                                     ...   
2021          1             THE PHOENIX PARTNERSHIP                      MICROTEST LTD                                                   59   
                                                                         THE PHOENIX PARTNERSHIP                                         17   
                                                                         Unknown                                                          5   
                            Unknown                                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                     0   
                                                                         THE PHOENIX PARTNERSHIP                                         27   

                                                                                                                                                 \
status                                                                                                                TransferStatus.INTEGRATED   
dateRequested dateRequested MName_requesting                             MName_sending                                                            
2020          9             EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                    151960   
                                                                         IN PRACTICE SYSTEMS LTD                                           2727   
                                                                         MICROTEST LTD                                                       18   
                                                                         THE PHOENIX PARTNERSHIP                                          47232   
                                                                         Unknown                                                            878   
...                                                                                                                                         ...   
2021          1             THE PHOENIX PARTNERSHIP                      MICROTEST LTD                                                        5   
                                                                         THE PHOENIX PARTNERSHIP                                              0   
                                                                         Unknown                                                             26   
                            Unknown                                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                        46   
                                                                         THE PHOENIX PARTNERSHIP                                             23   

                                                                

In [8]:
EMIS_name = "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)"
EMIS_monthly_transfer_breakdown = monthy_transfer_breakdown_by_supplier_pathways.iloc[monthy_transfer_breakdown_by_supplier_pathways.index.get_level_values('MName_requesting') == EMIS_name]
EMIS_monthly_transfer_breakdown=EMIS_monthly_transfer_breakdown['dateRequested']

In [9]:
EMIS_monthly_transfer_breakdown.div(EMIS_monthly_transfer_breakdown.sum(axis=1),axis=0)*100

status                                                                                                                 TransferStatus.FAILED  \
dateRequested dateRequested MName_requesting                             MName_sending                                                         
2020          9             EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)               1.311310   
                                                                         IN PRACTICE SYSTEMS LTD                                    1.271372   
                                                                         MICROTEST LTD                                             39.024390   
                                                                         THE PHOENIX PARTNERSHIP                                    7.959060   
                                                                         Unknown                                                   10.216110   
              10            EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)               1.433007   
                                                                         IN PRACTICE SYSTEMS LTD                                    1.007625   
                                                                         MICROTEST LTD                                             16.379310   
                                                                         THE PHOENIX PARTNERSHIP                                    8.693994   
                                                                         Unknown                                                    6.314244   
              11            EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)               1.324219   
                                                                         IN PRACTICE SYSTEMS LTD                                    0.534938   
                                                                         MICROTEST LTD                                             19.266055   
                                                                         THE PHOENIX PARTNERSHIP                                    9.573137   
                                                                         Unknown                                                    5.535055   
              12            EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)               1.191558   
                                                                         IN PRACTICE SYSTEMS LTD                                    0.903023   
                                                                         MICROTEST LTD                                             28.985507   
                                                                         THE PHOENIX PARTNERSHIP                                   10.425979   
                                                                         Unknown                                                    5.747126   
2021          1             EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)               1.435048   
                                                                         IN PRACTICE SYSTEMS LTD                                    1.580278   
                                                                         MICROTEST LTD                                             37.333333   
                                                                         THE PHOENIX PARTNERSHIP                                    9.197473   
                                                                         Unknown                                                    1.408451   

status                                                                                                         

In [11]:
import paths, data
response_codes = pd.read_csv(data.gp2gp_response_codes.path)
supplier_transfers = supplier_transfers.merge(response_codes, left_on='finalErrorCode',right_on='ErrorCode',how='left')

In [12]:
final_errors_per_suplier_path = supplier_transfers[
    ["MName_requesting", "MName_sending", "finalErrorCode"]
].pivot_table(
    index=[
        "MName_requesting", "MName_sending",
    ],
    columns=["finalErrorCode"],
    aggfunc=len,
    fill_value=0
)
final_errors_per_suplier_path

finalErrorCode                                                                             9.0   \
MName_requesting                             MName_sending                                        
EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)     0   
                                             IN PRACTICE SYSTEMS LTD                          0   
                                             MICROTEST LTD                                    0   
                                             THE PHOENIX PARTNERSHIP                          0   
                                             Unknown                                          0   
IN PRACTICE SYSTEMS LTD                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)     1   
                                             IN PRACTICE SYSTEMS LTD                          0   
                                             THE PHOENIX PARTNERSHIP                          0   
MICROTEST LTD                                EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)     0   
                                             THE PHOENIX PARTNERSHIP                          0   
THE PHOENIX PARTNERSHIP                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)     4   
                                             IN PRACTICE SYSTEMS LTD                          0   
                                             Unknown                                          0   
Unknown                                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)     0   
                                             IN PRACTICE SYSTEMS LTD                          0   
                                             THE PHOENIX PARTNERSHIP                          0   
                                             Unknown                                          0   

finalErrorCode                                                                             11.0  \
MName_requesting                             MName_sending                                        
EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)   240   
                                             IN PRACTICE SYSTEMS LTD                          9   
                                             MICROTEST LTD                                    0   
                                             THE PHOENIX PARTNERSHIP                         64   
                                             Unknown                                          1   
IN PRACTICE SYSTEMS LTD                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)    57   
                                             IN PRACTICE SYSTEMS LTD                          0   
                                             THE PHOENIX PARTNERSHIP                          8   
MICROTEST LTD                                EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)     6   
                                             THE PHOENIX PARTNERSHIP                          1   
THE PHOENIX PARTNERSHIP                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)   349   
                                             IN PRACTICE SYSTEMS LTD                          1   
                                             Unknown                                          4   
Unknown                                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)     1   
                                             IN PRACTICE SYSTEMS LTD                          0   
                                             THE PHOENIX PARTNERSHIP                          3   
                                             Unknown                                          0   

finalErrorCode                                                                             12.0  \
MName_requesting                             MName_sending                                        
EGTON MEDICAL INFORMATION SYSTEMS LTD 

In [17]:
final_errors_per_suplier_path = supplier_transfers[
    ["MName_requesting", "MName_sending", "ErrorName", "status"]
].pivot_table(
    index=[
        "MName_requesting", "MName_sending", 
    ],
    columns=["status", "ErrorName"],
    aggfunc=len,
    fill_value=0
)
final_errors_per_suplier_path

status                                                                                    TransferStatus.FAILED  \
ErrorName                                                                                      ABA wrong patint   
MName_requesting                             MName_sending                                                        
EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                   481   
                                             IN PRACTICE SYSTEMS LTD                                          3   
                                             MICROTEST LTD                                                    0   
                                             THE PHOENIX PARTNERSHIP                                        146   
                                             Unknown                                                          1   
IN PRACTICE SYSTEMS LTD                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                     0   
                                             IN PRACTICE SYSTEMS LTD                                          0   
                                             THE PHOENIX PARTNERSHIP                                          0   
MICROTEST LTD                                EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                     0   
                                             THE PHOENIX PARTNERSHIP                                          0   
THE PHOENIX PARTNERSHIP                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                    20   
                                             IN PRACTICE SYSTEMS LTD                                          0   
                                             Unknown                                                          0   
Unknown                                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)                     7   
                                             IN PRACTICE SYSTEMS LTD                                          0   
                                             THE PHOENIX PARTNERSHIP                                          6   
                                             Unknown                                                          1   

status                                                                                                   \
ErrorName                                                                                 Duplicate EHR   
MName_requesting                             MName_sending                                                
EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS) EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)          1342   
                                             IN PRACTICE SYSTEMS LTD                                  1   
                                             MICROTEST LTD                                            0   
                                             THE PHOENIX PARTNERSHIP                                123   
                                             Unknown                                                 14   
IN PRACTICE SYSTEMS LTD                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)           167   
                                             IN PRACTICE SYSTEMS LTD                                  1   
                                             THE PHOENIX PARTNERSHIP                                  1   
MICROTEST LTD                                EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)             0   
                                             THE PHOENIX PARTNERSHIP                                  0   
THE PHOENIX PARTNERSHIP                      EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)          3792   
                                             IN PRACTICE SYSTEMS LTD                                  1   
                                             Unknown                                          